In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 5.0 MB/s eta 0:00:00


In [3]:
!pip install catboost


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 6.6 MB/s eta 0:00:00


In [4]:
!pip install scikeras

In [5]:
# Import libraries
import optuna
import pandas as pd
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

# Load datasets
main_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/1_AAC/aac_main_positive.csv")
main_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/1_AAC/aac_main_negative (2).csv")
validation_p = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/1_AAC/aac_validation_positive (1).csv")
validation_n = pd.read_csv("/content/drive/MyDrive/Cell penetrating peptide/NEW_WORK/work_1/1_AAC/aac_validation_negative (1).csv")

# Prepare features and labels for training
X_train = pd.concat([main_p, main_n])
y_train = pd.concat([pd.Series(1, index=main_p.index), pd.Series(0, index=main_n.index)])


/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [6]:
# Define a cross-validation accuracy function
def cross_val_accuracy(model, X_train, y_train):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    accuracies = cross_val_score(model, X_train, y_train, cv=cv, scoring='accuracy')
    return accuracies.mean()


In [7]:
#Support Vector Machine (SVM)
def objective_svm(trial):
    kernel = trial.suggest_categorical('kernel', ['linear', 'rbf', 'poly'])
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)

    model = SVC(kernel=kernel, C=C, gamma=gamma)
    return cross_val_accuracy(model, X_train, y_train)

study_svm = optuna.create_study(direction='maximize')
study_svm.optimize(objective_svm, n_trials=50)
print("SVM Best Parameters:", study_svm.best_params)
print("SVM Best Accuracy:", study_svm.best_value)


[I 2024-12-09 22:07:10,109] A new study created in memory with name: no-name-a4f43601-1415-407b-ab90-af72751ade86
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
<ipython-input-7-cd4fae0f87b1>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  gamma = trial.suggest_loguniform('gamma', 1e-5, 1e-1)
[I 2024-12-09 22:07:10,390] Trial 0 finished with value: 0.5146625721474027 and parameters: {'kernel': 'linear', 'C': 0.0003767386144579884, 'gamma': 0.03380031668198025}. Best is trial 0 with value: 0.5146625721474027.
<ipython-input-7-cd4fae0f87b1>:4: FutureWarning: suggest_loguniform has been deprecat

SVM Best Parameters: {'kernel': 'poly', 'C': 89968.866455002, 'gamma': 0.07574790559793372}
SVM Best Accuracy: 0.8522236199496819


In [8]:
#Random Forest
def objective_rf(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_rf = optuna.create_study(direction='maximize')
study_rf.optimize(objective_rf, n_trials=50)
print("Random Forest Best Parameters:", study_rf.best_params)
print("Random Forest Best Accuracy:", study_rf.best_value)


[I 2024-12-09 22:09:03,373] A new study created in memory with name: no-name-478f920b-602d-4e84-b5ad-983d55c3aa0f
[I 2024-12-09 22:09:04,407] Trial 0 finished with value: 0.8968884120171673 and parameters: {'n_estimators': 51, 'max_depth': 10, 'min_samples_split': 19, 'min_samples_leaf': 2}. Best is trial 0 with value: 0.8968884120171673.
[I 2024-12-09 22:09:07,953] Trial 1 finished with value: 0.9149400621577624 and parameters: {'n_estimators': 173, 'max_depth': 19, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 1 with value: 0.9149400621577624.
[I 2024-12-09 22:09:09,687] Trial 2 finished with value: 0.8822850377386414 and parameters: {'n_estimators': 128, 'max_depth': 9, 'min_samples_split': 15, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.9149400621577624.
[I 2024-12-09 22:09:12,407] Trial 3 finished with value: 0.8977467811158798 and parameters: {'n_estimators': 140, 'max_depth': 24, 'min_samples_split': 18, 'min_samples_leaf': 5}. Best is trial 1 with valu

Random Forest Best Parameters: {'n_estimators': 211, 'max_depth': 14, 'min_samples_split': 3, 'min_samples_leaf': 1}
Random Forest Best Accuracy: 0.9321222435992305


In [9]:
#Decision Tree
def objective_dt(trial):
    max_depth = trial.suggest_int('max_depth', 3, 30)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 20)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

    model = DecisionTreeClassifier(
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf
    )
    return cross_val_accuracy(model, X_train, y_train)

study_dt = optuna.create_study(direction='maximize')
study_dt.optimize(objective_dt, n_trials=50)
print("Decision Tree Best Parameters:", study_dt.best_params)
print("Decision Tree Best Accuracy:", study_dt.best_value)


[I 2024-12-09 22:11:31,973] A new study created in memory with name: no-name-9377e815-14b9-4add-85f1-a36c80632fca
[I 2024-12-09 22:11:32,038] Trial 0 finished with value: 0.8298690247151101 and parameters: {'max_depth': 16, 'min_samples_split': 13, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.8298690247151101.
[I 2024-12-09 22:11:32,106] Trial 1 finished with value: 0.8539440580139116 and parameters: {'max_depth': 18, 'min_samples_split': 14, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8539440580139116.
[I 2024-12-09 22:11:32,174] Trial 2 finished with value: 0.8685548320260471 and parameters: {'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.8685548320260471.
[I 2024-12-09 22:11:32,234] Trial 3 finished with value: 0.8264540476542844 and parameters: {'max_depth': 22, 'min_samples_split': 11, 'min_samples_leaf': 10}. Best is trial 2 with value: 0.8685548320260471.
[I 2024-12-09 22:11:32,303] Trial 4 finished with value: 0

Decision Tree Best Parameters: {'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 3}
Decision Tree Best Accuracy: 0.8788663608110108


In [10]:
#k-Nearest Neighbors (k-NN)
def objective_knn(trial):
    n_neighbors = trial.suggest_int('n_neighbors', 3, 15)
    weights = trial.suggest_categorical('weights', ['uniform', 'distance'])
    algorithm = trial.suggest_categorical('algorithm', ['auto', 'ball_tree', 'kd_tree', 'brute'])

    model = KNeighborsClassifier(
        n_neighbors=n_neighbors,
        weights=weights,
        algorithm=algorithm
    )
    return cross_val_accuracy(model, X_train, y_train)

study_knn = optuna.create_study(direction='maximize')
study_knn.optimize(objective_knn, n_trials=50)
print("k-NN Best Parameters:", study_knn.best_params)
print("k-NN Best Accuracy:", study_knn.best_value)


[I 2024-12-09 22:11:37,425] A new study created in memory with name: no-name-9a05f209-b83e-4ae5-a0a0-7cb33370e5ba
[I 2024-12-09 22:11:37,692] Trial 0 finished with value: 0.9029302945093975 and parameters: {'n_neighbors': 4, 'weights': 'uniform', 'algorithm': 'brute'}. Best is trial 0 with value: 0.9029302945093975.
[I 2024-12-09 22:11:37,874] Trial 1 finished with value: 0.8857555128015392 and parameters: {'n_neighbors': 5, 'weights': 'uniform', 'algorithm': 'auto'}. Best is trial 0 with value: 0.9029302945093975.
[I 2024-12-09 22:11:37,959] Trial 2 finished with value: 0.9261284593754624 and parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'auto'}. Best is trial 2 with value: 0.9261284593754624.
[I 2024-12-09 22:11:38,176] Trial 3 finished with value: 0.8702937694243008 and parameters: {'n_neighbors': 7, 'weights': 'uniform', 'algorithm': 'kd_tree'}. Best is trial 2 with value: 0.9261284593754624.
[I 2024-12-09 22:11:38,250] Trial 4 finished with value: 0.9183957377

k-NN Best Parameters: {'n_neighbors': 4, 'weights': 'distance', 'algorithm': 'auto'}
k-NN Best Accuracy: 0.9261284593754624


In [11]:
#Logistic Regression

def objective_lr(trial):
    C = trial.suggest_loguniform('C', 1e-5, 1e5)
    penalty = trial.suggest_categorical('penalty', ['l2'])
    solver = trial.suggest_categorical('solver', ['lbfgs', 'saga'])

    model = LogisticRegression(C=C, penalty=penalty, solver=solver)
    return cross_val_accuracy(model, X_train, y_train)

study_lr = optuna.create_study(direction='maximize')
study_lr.optimize(objective_lr, n_trials=50)
print("Logistic Regression Best Parameters:", study_lr.best_params)
print("Logistic Regression Best Accuracy:", study_lr.best_value)


[I 2024-12-09 22:11:42,320] A new study created in memory with name: no-name-55d77766-da93-4818-93f9-b7b11da6eea1
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-09 22:11:42,394] Trial 0 finished with value: 0.7310714814266687 and parameters: {'C': 0.000735801494545141, 'penalty': 'l2', 'solver': 'lbfgs'}. Best is trial 0 with value: 0.7310714814266687.
<ipython-input-11-5e81b06dce9f>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform('C', 1e-5, 1e5)
[I 2024-12-09 22:11:42,505] Trial 1 finished with value: 0.78176705638597 and parameters: {'C': 45021.38759

Logistic Regression Best Parameters: {'C': 9.891075035049665, 'penalty': 'l2', 'solver': 'saga'}
Logistic Regression Best Accuracy: 0.7834874944501997


In [12]:
#Naive Bayes
def objective_nb(trial):
    var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)

    model = GaussianNB(var_smoothing=var_smoothing)
    return cross_val_accuracy(model, X_train, y_train)

study_nb = optuna.create_study(direction='maximize')
study_nb.optimize(objective_nb, n_trials=50)
print("Naive Bayes Best Parameters:", study_nb.best_params)
print("Naive Bayes Best Accuracy:", study_nb.best_value)


[I 2024-12-09 22:11:48,885] A new study created in memory with name: no-name-571b11c9-4ba1-4852-959a-09dd7163e9c9
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-09 22:11:48,979] Trial 0 finished with value: 0.7070297469291107 and parameters: {'var_smoothing': 8.159869954808957e-05}. Best is trial 0 with value: 0.7070297469291107.
<ipython-input-12-5c7e648b4d2b>:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  var_smoothing = trial.suggest_loguniform('var_smoothing', 1e-9, 1e-1)
[I 2024-12-09 22:11:49,043] Trial 1 finished with value: 0.7070297469291107 and 

Naive Bayes Best Parameters: {'var_smoothing': 0.09841187855293422}
Naive Bayes Best Accuracy: 0.7413793103448276


In [13]:
#Gradient Boosting
def objective_gb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = GradientBoostingClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate,
        max_depth=max_depth,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_gb = optuna.create_study(direction='maximize')
study_gb.optimize(objective_gb, n_trials=50)
print("Gradient Boosting Best Parameters:", study_gb.best_params)
print("Gradient Boosting Best Accuracy:", study_gb.best_value)


[I 2024-12-09 22:11:52,426] A new study created in memory with name: no-name-27ebc524-e18b-476c-ac98-da6b4f8d0b48
<ipython-input-13-0f44ec352f28>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-13-0f44ec352f28>:6: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-09 22:12:12,264] Trial 0 finished with value: 0.9140890927926595 and parameters: {'n_estimators': 158, 'learning_rate': 0.02350110406529668, 'max_depth': 12, 'subsample': 0.714084799872176}. Best is trial 0 with value: 0.9140890927926595.
<ipython-input-13-0f44ec352f28>:4: FutureWa

Gradient Boosting Best Parameters: {'n_estimators': 139, 'learning_rate': 0.05891017356326558, 'max_depth': 9, 'subsample': 0.5977855095202365}
Gradient Boosting Best Accuracy: 0.9218144146810715


In [14]:
#XGBoost

def objective_xgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    subsample = trial.suggest_uniform('subsample', 0.5, 1.0)

    model = XGBClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample
    )
    return cross_val_accuracy(model, X_train, y_train)

study_xgb = optuna.create_study(direction='maximize')
study_xgb.optimize(objective_xgb, n_trials=50)
print("XGBoost Best Parameters:", study_xgb.best_params)
print("XGBoost Best Accuracy:", study_xgb.best_value)


[I 2024-12-09 22:21:54,002] A new study created in memory with name: no-name-072aba09-4a27-447f-b6d2-ac3a60bfee68
<ipython-input-14-ad894fc3f237>:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
<ipython-input-14-ad894fc3f237>:7: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  subsample = trial.suggest_uniform('subsample', 0.5, 1.0)
[I 2024-12-09 22:21:54,737] Trial 0 finished with value: 0.7963889299985201 and parameters: {'n_estimators': 120, 'max_depth': 4, 'learning_rate': 3.577962104927119e-05, 'subsample': 0.8980183361127684}. Best is trial 0 with value: 0.7963889299985201.
<ipython-input-14-ad894fc3f237>:6: Future

XGBoost Best Parameters: {'n_estimators': 147, 'max_depth': 8, 'learning_rate': 0.09609991291368136, 'subsample': 0.5663142408254656}
XGBoost Best Accuracy: 0.923538552612106


In [15]:
# LightGBM
def objective_lgb(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    max_depth = trial.suggest_int('max_depth', 3, 20)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    num_leaves = trial.suggest_int('num_leaves', 2, 50)

    model = LGBMClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        learning_rate=learning_rate,
        num_leaves=num_leaves
    )
    return cross_val_accuracy(model, X_train, y_train)

study_lgb = optuna.create_study(direction='maximize')
study_lgb.optimize(objective_lgb, n_trials=50)
print("LightGBM Best Parameters:", study_lgb.best_params)
print("LightGBM Best Accuracy:", study_lgb.best_value)


[I 2024-12-09 22:23:26,571] A new study created in memory with name: no-name-faad94a2-380a-4e36-a9f3-2b532ae6da27
<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000775 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 871
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2024-12-09 22:23:27,007] Trial 0 finished with value: 0.7757140742933254 and parameters: {'n_estimators': 65, 'max_depth': 4, 'learning_rate': 0.0010279170750767336, 'num_leaves': 18}. Best is trial 0 with value: 0.7757140742933254.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


...


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No 

<ipython-input-15-f97698e6c3a7>:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)


[LightGBM] [Info] Number of positive: 465, number of negative: 466
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000443 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 871
[LightGBM] [Info] Number of data points in the train set: 931, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499463 -> initscore=-0.002148
[LightGBM] [Info] Start training from score -0.002148
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf

[I 2024-12-09 22:24:06,966] Trial 49 finished with value: 0.8470549060233832 and parameters: {'n_estimators': 84, 'max_depth': 11, 'learning_rate': 0.005034391837416744, 'num_leaves': 45}. Best is trial 22 with value: 0.9183920378866361.


[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [16]:
#AdaBoost
def objective_ab(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 200)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)

    model = AdaBoostClassifier(
        n_estimators=n_estimators,
        learning_rate=learning_rate
    )
    return cross_val_accuracy(model, X_train, y_train)

study_ab = optuna.create_study(direction='maximize')
study_ab.optimize(objective_ab, n_trials=50)
print("AdaBoost Best Parameters:", study_ab.best_params)
print("AdaBoost Best Accuracy:", study_ab.best_value)


[I 2024-12-09 22:24:06,995] A new study created in memory with name: no-name-7076a79d-634b-4f7b-9860-b4ec8d4a728c
<ipython-input-16-95971a9e6870>:4: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: Futu

AdaBoost Best Parameters: {'n_estimators': 170, 'learning_rate': 0.09332288953406956}
AdaBoost Best Accuracy: 0.8230057717922155


In [17]:
#Neural Network

import optuna
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler

# Load your dataset
# Replace this with your actual data loading process
X = np.random.rand(1000, 20)  # Dummy features
y = np.random.randint(0, 2, 1000)  # Dummy binary labels

# Normalize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_nn(trial):
    # Define hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 10, 50)

    # Build the neural network
    model = Sequential()
    model.add(Dense(hidden_units, activation='relu', input_shape=(X.shape[1],)))
    model.add(Dropout(dropout_rate))

    for _ in range(num_layers - 1):
        model.add(Dense(hidden_units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer=Adam(learning_rate=learning_rate),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    # Train the model (without validation data)
    history = model.fit(X, y,
                        batch_size=batch_size,
                        epochs=epochs,
                        verbose=0)

    # Evaluate the model
    train_accuracy = max(history.history['accuracy'])
    return train_accuracy

# Run Optuna study
study_nn = optuna.create_study(direction='maximize')
study_nn.optimize(objective_nn, n_trials=50)

# Print the best parameters and accuracy
print("Neural Network Best Parameters:", study_nn.best_params)
print("Neural Network Best Accuracy:", study_nn.best_value)

[I 2024-12-09 22:26:35,449] A new study created in memory with name: no-name-5018e10c-af79-4b41-bda0-c54bb2556e6e
<ipython-input-17-565ccca4d2fe>:23: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
<ipython-input-17-565ccca4d2fe>:24: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activit

Neural Network Best Parameters: {'num_layers': 3, 'hidden_units': 448, 'dropout_rate': 0.13155536977640367, 'learning_rate': 0.0002886226086550056, 'batch_size': 32, 'epochs': 47}
Neural Network Best Accuracy: 0.9940000176429749


In [18]:
#Multi-Layer Perceptron (MLP)

import optuna
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification

# Load your dataset (replace this with your actual dataset)
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Normalize the dataset
scaler = StandardScaler()
X = scaler.fit_transform(X)

def objective_mlp(trial):
    # Define the hyperparameters
    num_layers = trial.suggest_int('num_layers', 1, 3)
    hidden_units = trial.suggest_int('hidden_units', 32, 512, step=32)
    activation = trial.suggest_categorical('activation', ['relu', 'tanh', 'logistic'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
    batch_size = trial.suggest_categorical('batch_size', [16, 32, 64, 128])
    epochs = trial.suggest_int('epochs', 50, 200)

    # Define the MLP model
    model = MLPClassifier(
        hidden_layer_sizes=(hidden_units,) * num_layers,
        activation=activation,
        learning_rate_init=learning_rate,
        max_iter=epochs,
        batch_size=batch_size,
        solver='adam',
        random_state=42
    )

    # Evaluate using cross-validation
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    return scores.mean()

# Run Optuna study
study_mlp = optuna.create_study(direction='maximize')
study_mlp.optimize(objective_mlp, n_trials=50)

# Print results
print("MLP Best Parameters:", study_mlp.best_params)
print("MLP Best Accuracy:", study_mlp.best_value)

[I 2024-12-09 22:41:28,570] A new study created in memory with name: no-name-e96bafab-857b-4b23-b5bb-86a0b1a9bb34
<ipython-input-18-5ee0db595401>:21: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-2)
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (95) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (95) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Sto

MLP Best Parameters: {'num_layers': 3, 'hidden_units': 352, 'activation': 'logistic', 'learning_rate': 0.00020490349127639534, 'batch_size': 16, 'epochs': 137}
MLP Best Accuracy: 0.8760000000000001


probabilites dataset

In [19]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# Combine data and create labels
X_train = pd.concat([main_p, main_n], axis=0)
y_train = np.array([1] * len(main_p) + [0] * len(main_n))

X_validation = pd.concat([validation_p, validation_n], axis=0)
y_validation = np.array([1] * len(validation_p) + [0] * len(validation_n))

# Define models
models = {
    "RandomForest": RandomForestClassifier(random_state=42),
    "GradientBoosting": GradientBoostingClassifier(random_state=42),
    "LogisticRegression": LogisticRegression(max_iter=1000, random_state=42),
    "DecisionTree": DecisionTreeClassifier(random_state=42),
    "NaiveBayes": GaussianNB(),
    "k-NN": KNeighborsClassifier(),
    "SVM": SVC(probability=True, random_state=42),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42),
    "LightGBM": LGBMClassifier(random_state=42),
    #"CatBoost": CatBoostClassifier(verbose=0, random_state=42),
    "AdaBoost": AdaBoostClassifier(random_state=42),
    "MLP": MLPClassifier(max_iter=500, random_state=42)
}

# Combined probabilities dataset
combined_probabilities = pd.DataFrame(index=X_validation.index)

# Train models and get probabilities
for model_name, model in models.items():
    print(f"Training {model_name}...")
    model.fit(X_train, y_train)
    probabilities = model.predict_proba(X_validation)[:, 1]
    combined_probabilities[model_name] = probabilities

# Neural Network
def build_nn(input_dim):
    model = Sequential([
        Dense(128, activation='relu', input_dim=input_dim),
        Dropout(0.3),
        Dense(64, activation='relu'),
        Dropout(0.3),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Train Neural Network
input_dim = X_train.shape[1]
nn_model = build_nn(input_dim)
nn_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_validation, y_validation), verbose=1)

# Predict probabilities for Neural Network
nn_probabilities = nn_model.predict(X_validation).flatten()
combined_probabilities['NeuralNetwork'] = nn_probabilities

# Add ground truth
combined_probabilities['True_Label'] = y_validation

# Save the combined probabilities dataset
output_path = "AAC_optuna.csv"
combined_probabilities.to_csv(output_path, index=False)
print(f"Combined probabilities dataset with Neural Network saved to {output_path}")


Training RandomForest...
Training GradientBoosting...
Training LogisticRegression...
Training DecisionTree...
Training NaiveBayes...
Training k-NN...
Training SVM...
Training XGBoost...


/usr/local/lib/python3.10/dist-packages/xgboost/core.py:158: UserWarning: [00:04:27] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM...
[LightGBM] [Info] Number of positive: 582, number of negative: 582
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000774 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 966
[LightGBM] [Info] Number of data points in the train set: 1164, number of used features: 20
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Training AdaBoost...


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Training MLP...


/usr/local/lib/python3.10/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:690: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - accuracy: 0.6328 - loss: 0.6816 - val_accuracy: 0.7733 - val_loss: 0.6309
Epoch 2/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7723 - loss: 0.6176 - val_accuracy: 0.7667 - val_loss: 0.5315
Epoch 3/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7761 - loss: 0.5223 - val_accuracy: 0.8033 - val_loss: 0.4962
Epoch 4/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7696 - loss: 0.4953 - val_accuracy: 0.7967 - val_loss: 0.4616
Epoch 5/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7809 - loss: 0.4742 - val_accuracy: 0.8167 - val_loss: 0.4595
Epoch 6/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8039 - loss: 0.4367 - val_accuracy: 0.8167 - val_loss: 0.4412
Epoch 7/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8083 - loss: 0.4358 - val_accuracy: 0.8233 - val_loss: 0.4198
Epoch 8/50
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8214 - loss: 0.4067 - val_accuracy: 0.8267 - val_loss